In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os

# Use a clean modern style
plt.style.use('seaborn-v0_8-whitegrid')

# Set random seed for reproducibility
np.random.seed(0)

# Setup
T = 50
channels = 2
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# Generate time series data
true_series = np.cumsum(np.random.randn(T, channels), axis=0)

# Simulate sparse observations
mask = np.random.rand(T, channels) > 0.7
observed = true_series.copy()
observed[mask] = np.nan

# Simulated model outputs (mean and std)
imputed_mean = np.where(np.isnan(observed), true_series + np.random.normal(0, 0.1, size=(T, channels)), observed)
imputed_std = np.where(np.isnan(observed), 0.6 + 0.2 * np.random.rand(T, channels), 0)

# Color settings
obs_color = "#444444"
mean_color = ["#1f77b4", '#FF4500']
fill_color = ["#1f77b4", '#FF4500']

# Frame generation
filenames = []
for t in range(1, T + 1):
    fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

    for c in range(channels):
        ax = axs[c]
        
        # Plot all observed points
        observed_mask = ~np.isnan(observed[:, c])
        ax.plot(np.where(observed_mask)[0], observed[observed_mask, c],
                'o', color=obs_color, label="Observed" if t == 1 and c == 0 else "", markersize=5)

        # Plot imputed mean
        ax.plot(range(t), imputed_mean[:t, c], '-', color=mean_color[c],
                label="Mean Imputation" if t == 1 and c == 0 else "", linewidth=2)

        # Plot uncertainty band
        upper = imputed_mean[:t, c] + imputed_std[:t, c]
        lower = imputed_mean[:t, c] - imputed_std[:t, c]
        ax.fill_between(range(t), lower, upper, color=fill_color[c], alpha=0.3,
                        label="Uncertainty" if t == 1 and c == 0 else "")

        ax.set_ylabel(f'Channel {c+1}', fontsize=12)
        ax.tick_params(labelsize=10)
        ax.set_xlim(0, T - 1)
        ax.set_ylim(true_series.min() - 1, true_series.max() + 1)

        if c == 0 and t == 1:
            ax.legend(loc="upper left", fontsize=10)

    axs[-1].set_xlabel("Time", fontsize=12)
    # fig.suptitle("Probabilistic Multi-Channel Imputation", fontsize=15, weight='bold')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Save frame
    filename = f"{output_dir}/frame_{t:03d}.png"
    plt.savefig(filename, dpi=100)
    plt.close()
    filenames.append(filename)

# Create GIF
with imageio.get_writer("imputation_reveal_nice.gif", mode='I', duration=0.1, loop=0) as writer:
    for filename in filenames:
        writer.append_data(imageio.imread(filename))

# Clean up
for f in filenames:
    os.remove(f)
os.rmdir(output_dir)

print("GIF saved as 'imputation_reveal_nice.gif'")


GIF saved as 'imputation_reveal_nice.gif'
